THIS WAS THE FILE THAT WAS USED INITIALLY WHEN WE WERE TESTING OUT GCN FOR OUR BIN DATA BUT THIS ISNT THE FINAL FILE FOR GCN, THE GCNonProtienGraphs FILE IS FINAL FILE. THIS FILE WAS INCLUDED TO SHOW OUR EXPERIMENTS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/276/proj

Mounted at /content/drive
/content/drive/My Drive/276/proj


In [ ]:
!pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 48.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import dgl
from dgl.data.utils import split_dataset

import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.nn as dglnn

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
# Load the graphs and the labels
graphs, label_dict = dgl.load_graphs('/content/drive/My Drive/276/proj/graphs/dgl_graphs.bin')
labels = label_dict['labels']

In [ ]:
combined_dataset = list(zip(graphs, labels))
random.shuffle(combined_dataset)
split_index = int(len(combined_dataset) * 0.8)
trainset = combined_dataset[:split_index]
testset = combined_dataset[split_index:]

In [ ]:
class GCN(nn.Module):
  def __init__(self, in_feats, hidden_size, num_classes):
    super(GCN, self).__init__()
    self.conv1 = dglnn.GraphConv(in_feats, hidden_size)
    self.conv2 = dglnn.GraphConv(hidden_size, num_classes)

  def forward(self, g, features):
    x = F.relu(self.conv1(g, features))
    x = self.conv2(g, x)
    g.ndata['h'] = x
    return dgl.mean_nodes(g, 'h')

In [ ]:
def train(model, trainset, epochs, learning_rate):
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  model.train()
  for epoch in range(epochs):
    total_loss = 0
    for graph, label in trainset:
      features = graph.ndata['b_factor'].float()
      features = features.unsqueeze(1)
      logits = model(graph, features)
      loss = F.cross_entropy(logits, label.view(-1))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    avg_loss = total_loss / len(trainset)
    print(f"Epoch {epoch}, Loss: {avg_loss}")

In [ ]:
############s
def evaluate(model, testset):
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0
    for graph, label in testset:
      features = graph.ndata['b_factor'].float()
      features = features.unsqueeze(1)

      logits = model(graph, features)
      _, predicted = torch.max(logits.data, 1)
      total += 1  # Increment the total count by 1 for each graph
      correct += (predicted == label).sum().item()
  accuracy = 100 * correct / total
  return accuracy

In [ ]:
sample_graph = graphs[0]  # Take the first graph as a sample
sample_graph.ndata

In [ ]:
class GCN(nn.Module):
  def __init__(self, in_feats, hidden_size, num_classes, dropout_rate):
    super(GCN, self).__init__()
    self.conv1 = dglnn.GraphConv(in_feats, hidden_size)
    self.conv2 = dglnn.GraphConv(hidden_size, hidden_size)  # Additional layer
    self.conv3 = dglnn.GraphConv(hidden_size, num_classes)
    self.dropout = nn.Dropout(p=0.5)  # Dropout layer
  def forward(self, g, features):
    x = F.relu(self.conv1(g, features))
    x = self.dropout(x)
    x = F.relu(self.conv2(g, x))
    x = self.dropout(x)
    x = self.conv3(g, x)
    g.ndata['h'] = x
    return dgl.mean_nodes(g, 'h')

In [ ]:
"""
model = GCN(in_feats=1, hidden_size=256, num_classes=10)
train(model, trainset, epochs=10, learning_rate=0.0005)
accuracy = evaluate(model, testset)
print(f"Test accuracy: {accuracy}%")
"""

'\n# Initialize the model\nmodel = GCN(in_feats=1, hidden_size=256, num_classes=10)\n\n# Train the model with a smaller learning rate and more epochs\ntrain(model, trainset, epochs=10, learning_rate=0.0005)\n\n\n# Evaluate the model\naccuracy = evaluate(model, testset)\nprint(f"Test accuracy: {accuracy}%")\n'

In [ ]:
import itertools

learning_rates = [0.01, 0.001, 0.0001]
hidden_sizes = [16, 32, 64]
num_epochs = [10, 20, 30]
dropout_rates = [0.3, 0.5, 0.7]

results = []

for lr, hidden_size, epochs, dropout_rate in itertools.product(learning_rates, hidden_sizes, num_epochs, dropout_rates):
  model = GCN(in_feats=1, hidden_size=hidden_size, num_classes=10, dropout_rate=dropout_rate)
  train(model, trainset, epochs=epochs, learning_rate=lr)
  accuracy = evaluate(model, testset)
  results.append((lr, hidden_size, epochs, dropout_rate, accuracy))
  print(f"LR: {lr}, Hidden Size: {hidden_size}, Epochs: {epochs}, Dropout: {dropout_rate}, Accuracy: {accuracy}%")


best_hyperparams = max(results, key=lambda x: x[-1])
print("Best Hyperparameters:", best_hyperparams)

Epoch 0, Loss: 2.4080974222125864
Epoch 1, Loss: 2.3033141761196587
Epoch 2, Loss: 2.29706900505195
Epoch 3, Loss: 2.2967570938711477
Epoch 4, Loss: 2.2928391948230287
Epoch 5, Loss: 2.2839655247995427
Epoch 6, Loss: 2.2926002368144225
Epoch 7, Loss: 2.287236283895067
Epoch 8, Loss: 2.278917673611103
Epoch 9, Loss: 2.2750260602860224
LR: 0.01, Hidden Size: 16, Epochs: 10, Dropout: 0.3, Accuracy: 9.0%
Epoch 0, Loss: 2.4155713040428353
Epoch 1, Loss: 2.3211523521514166
Epoch 2, Loss: 2.3065444244477984
Epoch 3, Loss: 2.294940422725558
Epoch 4, Loss: 2.292730655362433
Epoch 5, Loss: 2.2779390229318377
Epoch 6, Loss: 2.2749097991109193
Epoch 7, Loss: 2.278762902532305
Epoch 8, Loss: 2.278726803255559
Epoch 9, Loss: 2.2814388754299113
LR: 0.01, Hidden Size: 16, Epochs: 10, Dropout: 0.5, Accuracy: 9.0%
Epoch 0, Loss: 2.6422569824937607
Epoch 1, Loss: 2.3112335021334482
Epoch 2, Loss: 2.3115309957872356
Epoch 3, Loss: 2.307743164456279
Epoch 4, Loss: 2.2991793515688195
Epoch 5, Loss: 2.301008

In [ ]:
def evaluate(model, testset):
  model.eval()
  dev_loss = 0
  predictions = []
  true_labels = []
  loss_criterion = torch.nn.CrossEntropyLoss()
  with torch.no_grad():
    for graph, label in testset:
      features = graph.ndata['b_factor'].float().unsqueeze(1)
      preds = model(graph, features)
      loss = loss_criterion(preds, label.view(-1))
      dev_loss += loss.item()

      _, pred_labels = torch.max(preds.data, 1)
      predictions.extend(pred_labels.numpy())
      true_labels.append(label.item())
  avg_dev_loss = dev_loss / len(testset)
  return avg_dev_loss, predictions, true_labels

In [ ]:
def train(model, trainset, epochs, learning_rate):
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  loss_criterion = torch.nn.CrossEntropyLoss()
  for epoch in range(1, epochs + 1):
    model.train()
    train_loss = 0
    train_predictions = []
    train_true_labels = []
    for graph, label in trainset:
      features = graph.ndata['b_factor'].float().unsqueeze(1)
      optimizer.zero_grad()
      preds = model(graph, features)
      loss = loss_criterion(preds, label.view(-1))
      train_loss += loss.item()
      _, pred_labels = torch.max(preds.data, 1)
      train_predictions.extend(pred_labels.numpy())
      train_true_labels.append(label.item())
      loss.backward()
      optimizer.step()
    avg_train_loss = train_loss / len(trainset)
    print(f"Epoch {epoch}/{epochs}, Train Loss: {avg_train_loss}")

In [ ]:
import itertools
learning_rates = [0.01, 0.001, 0.0001]
hidden_sizes = [16, 32, 64]
num_epochs = [10, 20, 30]
dropout_rates = [0.3, 0.5, 0.7]
results = []
for lr, hidden_size, epochs, dropout_rate in itertools.product(learning_rates, hidden_sizes, num_epochs, dropout_rates):
    model = GCN(in_feats=1, hidden_size=hidden_size, num_classes=10, dropout_rate=dropout_rate)
    train(model, trainset, epochs=epochs, learning_rate=lr)
    dev_loss, predictions, true_labels = evaluate(model, testset)
    accuracy = sum([pred == true for pred, true in zip(predictions, true_labels)]) / len(true_labels) * 100
    results.append((lr, hidden_size, epochs, dropout_rate, accuracy))
    print(f"LR: {lr}, Hidden Size: {hidden_size}, Epochs: {epochs}, Dropout: {dropout_rate}, Dev Loss: {dev_loss}, Accuracy: {accuracy}%")
best_hyperparams = max(results, key=lambda x: x[-1])
print("Best Hyperparameters:", best_hyperparams)

Epoch 1/10, Train Loss: 2.7611734471971787
Epoch 2/10, Train Loss: 2.3047965154014434
Epoch 3/10, Train Loss: 2.3100367403568183
Epoch 4/10, Train Loss: 2.30850056598061
Epoch 5/10, Train Loss: 2.309676281379578
Epoch 6/10, Train Loss: 2.3215335145928804
Epoch 7/10, Train Loss: 2.304690524300836
Epoch 8/10, Train Loss: 2.2957751179548134
Epoch 9/10, Train Loss: 2.2980519046162007
Epoch 10/10, Train Loss: 2.3012732164304057
LR: 0.01, Hidden Size: 16, Epochs: 10, Dropout: 0.3, Dev Loss: 2.336989666223526, Accuracy: 9.0%
Epoch 1/10, Train Loss: 2.541472768518411
Epoch 2/10, Train Loss: 2.298008372983837
Epoch 3/10, Train Loss: 2.3034818459274176
Epoch 4/10, Train Loss: 2.3224698025779915
Epoch 5/10, Train Loss: 2.295512059502733
Epoch 6/10, Train Loss: 2.2880758003035284
Epoch 7/10, Train Loss: 2.285045603612312
Epoch 8/10, Train Loss: 2.288876989655626
Epoch 9/10, Train Loss: 2.2805626732962474
Epoch 10/10, Train Loss: 2.280113892522372
LR: 0.01, Hidden Size: 16, Epochs: 10, Dropout: 0.5